In [8]:
%pip install torch summary wandb

  Preparing metadata (setup.py) ... done
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 1.9 MB/s eta 0:00:0000:0100:01
Using cached click-8.1.7-py3-none-any.whl (97 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 1.7 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 2.2 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 2.4 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.0 MB/s eta 0:00:0000:0100:01m
Using cached gitdb-4.0.11-py3-none-any.whl (62 kB)
Using cached smmap-5.0.1-py3-none-any.whl (24 kB)
  Created wheel for summary: filename=summary-0.2.0-py3-none-any.whl size=18125 sha256=2fc

In [10]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [7]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchsummary import summary

class H97_ResNet(nn.Module):
    def __init__(self, num_classes: int = 3):
        super(H97_ResNet, self).__init__()        
        # Tải mô hình ResNet50 đã được huấn luyện trước
        resnet50 = models.resnet50(pretrained=True)
        # Loại bỏ lớp fully connected cuối cùng
        self.feature_extractor = nn.Sequential(*list(resnet50.children())[:-1])
        # Đóng băng các tham số trong feature extractor để không cập nhật trong quá trình huấn luyện
        for param in self.feature_extractor.parameters():
            param.requires_grad = False

        # Kích thước đầu vào cho lớp fully connected đầu tiên dựa trên output của ResNet50
        # ResNet50 thường trả về tensor [batch_size, 2048, 1, 1] sau lớp pooling cuối cùng
        self.fc1 = nn.Linear(2048, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        # Trích xuất đặc trưng
        x = self.feature_extractor(x)
        # Chuyển đổi tensor từ [batch_size, 2048, 1, 1] sang [batch_size, 2048] để phù hợp với lớp fully connected
        x = torch.flatten(x, 1)
        # Đưa qua mạng dense
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        
        x = self.fc3(x)
        return x

# # Khởi tạo mô hình
# model = H97_ResNet()

# # Trực quan hóa kiến trúc mô hình
# summary(model, (3, 224, 224))  # Input shape (3 channels, 224x224 image)


/home/harito/venv/ai_env_3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/harito/venv/ai_env_3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [6]:
2048 *7

14336

In [2]:
%pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 682.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 234.3 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 189.6 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 229.9 kB/s eta 0:00:0000:0100:06

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

# Giả sử labels và predictions
# [[41, 14], [4, 666]]
labels = [1]*45 + [0]*680  # 41 TP + 4 FN, 14 FP + 666 TN
predictions = [1]*55 + [0]*670  # 41 TP + 14 FP, 4 FN + 666 TN

# Tính toán F1 Score và Accuracy
f1 = f1_score(labels, predictions)
accuracy = accuracy_score(labels, predictions)

print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

# Để tính ROC AUC, bạn cần có dự đoán dưới dạng xác suất thay vì nhãn cứng.
# roc_auc = roc_auc_score(labels, probability_predictions)

F1 Score: 0.9
Accuracy: 0.9862068965517241


In [16]:
import numpy as np
import os

def read_data():
    # Assuming you have the model_destination and model_name variables defined
    # model_destination = "path/to/your/model"
    # model_name = "your_model_name"

    # # Construct the file path
    # file_path = os.path.join(model_destination, f"test_{model_name}_metrics.npz")
    file_path = '/Data/Projects/ThyroidCancer_InfoData_Local/kaggle/working/h97_retrainEfficientNet_B2_B5_B6_dataver3/test_h97_retrainEfficientNet_B2_B5_B6_dataver3_metrics.npz'
    # Load the data
    data = np.load(file_path, allow_pickle=True)

    # Accessing the saved items
    test_preds = data['test_preds']
    test_probs = data['test_probs']
    test_targets = data['test_targets']
    test_loss = data['test_loss']
    test_acc = data['test_acc']
    test_f1 = data['test_f1']
    target_names = data['target_names']
    cm = data['cm']
    cr = data['cr']
    roc_auc = data['roc_auc']
    fpr = data['fpr']
    tpr = data['tpr']

    # Remember to close the file after loading the data
    data.close()
    return test_preds, test_targets 

In [17]:
test_preds, test_targets = read_data()
print(len(test_preds), test_preds)
print(len(test_targets), test_targets)

725 [0 2 0 2 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 2 1 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 1
 0 0 0 0 0 0 2 1 0 0 0 0 0 0 1 1 0 0 1 1 2 2 1 1 1 2 1 1 1 1 2 1 2 1 1 1 1
 1 2 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 2 1 1 2 1 1 1 1 1
 1 2 1 1 1 2 1 1 2 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 2 1 1 1 1 1 2 2 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 1 1 1 2 1 1 1 2 1 1 1 1 2 1 1 1 1
 1 1 1 1 1 2 1 1 1 1 2 1 1 2 0 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 2 1 1 1 1
 1 2 2 1 1 1 1 1 1 1 1 1 0 1 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 2 1 1 2 1
 2 1 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 2 1 1 2 1
 1 1 1 2 1 1 1 1 1 2 1 1 1 1 2 1 2 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 2 2 2
 2 2 2 2 1 2 1 2 2 1 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2
 2 1 2 1 2 2 2 2 2 1 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 1 2 1 2 1 2 2 1 2 2 2
 1 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 1 2 2 2 1 2 1 2 2 1 2 1 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 1 2 2 1 2
 2 2 2 1 2 2 2 2 2 2 

In [18]:
# Calculate F1 score and accuracy
f1 = f1_score(test_targets, test_preds, average='weighted')
accuracy = accuracy_score(test_targets, test_preds)
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

F1 Score: 0.8224652503383926
Accuracy: 0.8220689655172414


In [19]:
import numpy as np
from scipy.stats import mode
from sklearn.metrics import f1_score, accuracy_score

# Assuming test_preds and test_targets are numpy arrays with 725 elements each
# Reshape the predictions and targets to have 5 columns, corresponding to each object's predictions
reshaped_preds = test_preds.reshape(-1, 5)
reshaped_targets = test_targets.reshape(-1, 5)

# Determine the mode (most frequent label) for each group of predictions
mode_preds, _ = mode(reshaped_preds, axis=1)
mode_preds = mode_preds.flatten()

# Since all 5 targets for each object should be the same, we can just take one from each group
final_targets = reshaped_targets[:, 0]

# Calculate F1 score and accuracy
f1 = f1_score(final_targets, mode_preds, average='weighted')
accuracy = accuracy_score(final_targets, mode_preds)

print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

F1 Score: 0.98564039408867
Accuracy: 0.9862068965517241


In [10]:
print(len(mode_preds), mode_preds)
print(len(final_targets), final_targets)

145 [0 0 0 0 1 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
145 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [13]:
print(len(reshaped_preds), reshaped_preds)
print(len(reshaped_targets), reshaped_targets)

145 [[0 2 0 2 0]
 [0 0 1 0 1]
 [0 1 1 0 0]
 [0 0 0 0 2]
 [1 0 1 1 0]
 [0 0 0 0 0]
 [1 0 0 1 1]
 [1 1 0 0 0]
 [0 0 0 2 1]
 [0 0 0 0 0]
 [0 1 1 0 0]
 [1 1 2 2 1]
 [1 1 2 1 1]
 [1 1 2 1 2]
 [1 1 1 1 1]
 [2 2 1 1 1]
 [1 1 1 1 1]
 [2 1 1 1 1]
 [1 1 1 1 2]
 [1 1 1 1 1]
 [1 1 2 1 1]
 [2 1 1 1 1]
 [1 1 2 1 1]
 [1 2 1 1 2]
 [1 1 1 1 1]
 [1 1 1 1 2]
 [2 1 1 1 1]
 [1 1 2 1 1]
 [1 1 1 2 2]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 2]
 [1 1 2 1 1]
 [1 2 1 1 1]
 [2 1 1 1 1]
 [2 1 1 1 1]
 [1 1 1 1 1]
 [2 1 1 1 1]
 [2 1 1 2 0]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 2 1]
 [1 1 2 1 1]
 [1 1 1 2 2]
 [1 1 1 1 1]
 [1 1 1 1 0]
 [1 1 1 1 2]
 [1 1 1 1 1]
 [2 1 1 1 1]
 [1 1 1 1 2]
 [1 1 2 1 2]
 [1 2 1 1 1]
 [1 1 2 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 2 1]
 [1 1 1 1 1]
 [1 2 1 1 2]
 [1 1 1 1 2]
 [1 1 1 1 1]
 [2 1 1 1 1]
 [2 1 2 1 1]
 [1 1 1 1 2]
 [2 1 1 1 1]
 [1 1 1 1 1]
 [2 2 2 2 2]
 [2 2 1 2 1]
 [2 2 1 2 2]
 [2 2 2 2 2]
 [2 2 2 1 1]
 [2 2 2 1 2]
 [2 2 2 2 2]
 [2 2 2 2 2]
 [2 1 2 1 2]
 [2 2 2 2 1]
 [2 2 2 

---

In [20]:
import numpy as np
import os

def read_data():
    # Assuming you have the model_destination and model_name variables defined
    # model_destination = "path/to/your/model"
    # model_name = "your_model_name"

    # # Construct the file path
    # file_path = os.path.join(model_destination, f"test_{model_name}_metrics.npz")
    file_path = '/Data/Projects/ThyroidCancer_InfoData_Local/kaggle/working/final_test_with_transform_224x224/test_set/final_test_metrics.npz'
    file_path = '/home/harito/Downloads/test_h97_retrainEfficientNet_B2_B5_B6_dataver3_metrics.npz'
    # Load the data
    data = np.load(file_path, allow_pickle=True)

    # Accessing the saved items
    test_preds = data['test_preds']
    test_targets = data['test_targets']
    test_acc = data['test_acc']
    test_f1 = data['test_f1']
    target_names = data['target_names']
    cm = data['cm']
    cr = data['cr']

    # Remember to close the file after loading the data
    data.close()
    return test_preds, test_targets 

In [21]:
test_preds, test_targets = read_data()
print(len(test_preds), test_preds)
print(len(test_targets), test_targets)

725 [0 1 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 2 1 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 1
 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 1 1 1 2 2 1 1 1 2 1 1 1 1 2 1 1 1 1 1 1
 1 2 2 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1
 1 2 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 2 1 1 1 1 1 1 2 1 1 1
 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1
 1 1 1 1 1 2 1 1 1 1 2 1 1 2 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 2 1 1 1 1 1 1 1 1 2 0 1 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 2 1 1 2 1
 2 1 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 0 2 1 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 2 1 1 1 1 1 2 2 2
 2 2 2 2 1 2 1 2 2 1 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2
 1 2 2 1 2 2 2 2 2 1 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 1 2 1 2 1 2 2 2 2 2 2
 1 2 2 2 1 2 2 1 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 1 1 2 2 1 2 1 2 2 1 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 1 2 2 2 2 2 2 

In [22]:
import numpy as np
from scipy.stats import mode
from sklearn.metrics import f1_score, accuracy_score

# Assuming test_preds and test_targets are numpy arrays with 725 elements each
# Reshape the predictions and targets to have 5 columns, corresponding to each object's predictions
reshaped_preds = test_preds.reshape(-1, 5)
reshaped_targets = test_targets.reshape(-1, 5)

# Determine the mode (most frequent label) for each group of predictions
mode_preds, _ = mode(reshaped_preds, axis=1)
mode_preds = mode_preds.flatten()

# Since all 5 targets for each object should be the same, we can just take one from each group
final_targets = reshaped_targets[:, 0]

# Calculate F1 score and accuracy
f1 = f1_score(final_targets, mode_preds, average='weighted')
accuracy = accuracy_score(final_targets, mode_preds)

print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

F1 Score: 0.9721196891432826
Accuracy: 0.9724137931034482


In [35]:
import numpy as np
from scipy.stats import mode 
from sklearn.metrics import f1_score, accuracy_score

# Assuming test_preds and test_targets are numpy arrays with 725 elements each
# Reshape the predictions and targets to have 5 columns, corresponding to each object's predictions
reshaped_preds = test_preds.reshape(-1, 5)
reshaped_targets = test_targets.reshape(-1, 5)

# Determine the mode (most frequent label) for each group of predictions
mode_preds, _ = mode(reshaped_preds, axis=1)
# mode_preds = np.mean(reshaped_preds, axis=1)
mode_preds = mode_preds.flatten()
print(mode_preds)
mode_preds = np.round(mode_preds).astype(int)
print(mode_preds)
print(final_targets)
# Since all 5 targets for each object should be the same, we can just take one from each group
final_targets = reshaped_targets[:, 0]

# Calculate F1 score and accuracy
f1 = f1_score(final_targets, mode_preds, average='weighted')
accuracy = accuracy_score(final_targets, mode_preds)
# f1 = f1_score(mode_preds, final_targets, average='weighted')
# accuracy = accuracy_score(mode_preds, final_targets)

print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

[0 0 0 0 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
F1 Score: 0.9721196891432826
Accuracy: 0.9724137931034482


In [23]:
print(len(reshaped_preds), reshaped_preds)
print(len(reshaped_targets), reshaped_targets)

145 [[0 1 0 1 0]
 [0 0 1 0 1]
 [0 0 1 0 0]
 [0 0 0 1 2]
 [1 0 1 1 0]
 [0 0 0 0 0]
 [1 0 0 0 1]
 [0 1 0 0 0]
 [0 0 0 1 1]
 [0 0 0 0 0]
 [0 1 1 0 1]
 [1 1 2 2 1]
 [1 1 2 1 1]
 [1 1 2 1 1]
 [1 1 1 1 1]
 [2 2 1 1 1]
 [1 1 1 1 2]
 [2 1 1 1 1]
 [1 1 1 1 2]
 [1 1 1 1 1]
 [1 2 2 1 1]
 [1 1 1 1 1]
 [1 1 2 1 1]
 [1 1 1 1 1]
 [1 0 1 1 1]
 [1 1 1 1 2]
 [2 1 1 1 1]
 [1 1 2 1 1]
 [1 1 1 1 2]
 [1 1 1 1 2]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 2]
 [1 1 1 1 1]
 [1 2 1 1 1]
 [1 1 1 1 1]
 [2 1 1 1 1]
 [1 1 1 1 1]
 [2 1 1 1 1]
 [2 1 1 2 0]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 2]
 [1 1 1 1 1]
 [1 1 1 2 0]
 [1 1 1 1 2]
 [1 1 1 1 1]
 [2 1 1 1 1]
 [1 1 1 1 2]
 [1 1 2 1 2]
 [1 2 1 1 1]
 [1 1 2 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 2 1]
 [1 1 1 1 1]
 [0 2 1 1 2]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [2 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 2]
 [2 1 1 1 2]
 [1 1 1 1 1]
 [2 2 2 2 2]
 [2 2 1 2 1]
 [2 2 1 2 2]
 [2 2 2 2 2]
 [2 2 2 1 1]
 [2 2 2 1 2]
 [2 2 2 2 2]
 [2 2 2 2 2]
 [1 2 2 1 2]
 [2 2 2 2 1]
 [2 2 2 

In [37]:
from sklearn.metrics import accuracy_score

# Predicted labels
predicted = [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2
, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2
, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
# True labels
true = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2
, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2
, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
# Calculate accuracy
accuracy = accuracy_score(true, predicted)

print(f"Accuracy: {accuracy}")

Accuracy: 0.9724137931034482
